In [1]:
# pip install pyctcdecode
# pip install torch
# pip install pypi-kenlm

In [1]:
import torch
import torchaudio

from transformers import AutoModelForCTC, Wav2Vec2ProcessorWithLM
import torchaudio.utils
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = AutoModelForCTC.from_pretrained("bhuang/asr-wav2vec2-french").to(device)
processor_with_lm = Wav2Vec2ProcessorWithLM.from_pretrained("bhuang/asr-wav2vec2-french")
model_sample_rate = processor_with_lm.feature_extractor.sampling_rate

wav_path = "C:\\Users\\trist\\PycharmProjects\\Political_debate_summary\\data\\hemi16Janv2023\\hemi16Janv2023_presidente_de_seance_splits\\wav\\hemi16Janv2023_presidente_de_seance.wavout000.wav"  # path to your audio file
waveform, sample_rate = torchaudio.load(wav_path)
waveform = waveform.squeeze(axis=0)  # mono

# resample
if sample_rate != model_sample_rate:
    resampler = torchaudio.transforms.Resample(sample_rate, model_sample_rate)
    waveform = resampler(waveform)

# normalize
input_dict = processor_with_lm(waveform, sampling_rate=model_sample_rate, return_tensors="pt")

with torch.inference_mode():
    logits = model(input_dict.input_values.to(device)).logits

predicted_sentence = processor_with_lm.batch_decode(logits.cpu().numpy()).text[0]


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
